In [ ]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from train import init_model, init_training, create_connections_datasets

In [ ]:
tokenizer, model = init_model()

In [4]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [5]:
training_objects = init_training(model, train_dataset)

In [ ]:
from train import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_new_tokens=to.config.max_new_tokens,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

In [ ]:
episodes[0].reward

In [ ]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
    apply_loss=True
)


In [ ]:
results